In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

8489

In [3]:
!mkdir page

mkdir: cannot create directory ‘page’: File exists


In [8]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    success = False
    while True:
        try:
            r = requests.get('https://hellodoktor.com' + url)
            success = True
            break
        except Exception as e:
            if 'xceeded 30 redirects' in str(e):
                break
            time.sleep(1.0)
            
    if not success:
        continue
            
    soup = BeautifulSoup(r.content, "lxml")

    try:
        title = soup.find('h1').text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'class': 'unique-content-wrapper'}).get_text(separator="\n")
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

8489it [2:08:11,  1.10it/s]


In [7]:
'https://hellodoktor.com' + url

'https://hellodoktor.com/kehamilan/rancang-kehamilan/cabaran/tanda-lelaki-mandul/'

In [9]:
from glob import glob

files = glob('page/*.json')
len(files)

8488

In [10]:
with open('hellodoktor.com.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            d = json.load(fopen)
        fopen_l.write(f'{json.dumps(d)}\n')